# **Esercitazione 2 - Regressione Lineare**

## Boston Housing dataset

Questo dataset contiene informazioni raccolte dal U.S. Census Service riguardanti le abitazioni nell'area di Boston, Massachusetts. È stato ottenuto dall'archivio StatLib (http://lib.stat.cmu.edu/datasets/boston) ed è stato ampiamente utilizzato in letteratura per fare benchmark di algoritmi. 

Il dataset contiene informazioni su 506 case, divise in 14 variabili.

In [21]:
import numpy as np
import matplotlib.pyplot as plt

In [22]:
import pandas as pd 
from sklearn.utils import shuffle
from pandas import read_csv

from sklearn.datasets import fetch_openml
import pandas as pd

# Scarica il Boston Housing Dataset da OpenML
boston = fetch_openml(name="Boston", version=1, as_frame=True)

# Estrai i dati (features) e il target (valore mediano delle abitazioni)
X = boston.data
y = boston.target

X, y = shuffle(X, y, random_state=0)
print(f"Features shape: {X.shape}, targets shape:  {y.shape}")

Features shape: (506, 13), targets shape:  (506,)


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


## `np.c_` in NumPy

L'oggetto `np.c_` in NumPy è una **scorciatoia** per concatenare array lungo il secondo asse (cioè, le colonne).

## Utilizzo
```python
np.c_[array1, array2, ...]


In [23]:
import numpy as np

# Generate two random 2x3 matrices
matrice1 = np.random.rand(2, 3)
matrice2 = np.random.rand(2, 3)

# Concatenate the matrices along columns
risultato = np.c_[matrice1, matrice2]

print("Matrice 1:",matrice1.shape)

print("\nMatrice 2:",matrice2.shape)

print("\nMatrice concatenata:",risultato.shape)

Matrice 1: (2, 3)

Matrice 2: (2, 3)

Matrice concatenata: (2, 6)


**Divisione del dataset**

Il primo passaggio è quello di dividere i dati in train set, validation set e test set. Utilizza il 60% dei dati per il training set, il 20% per il validation e il restante 20% per il test set. Considerato che il nostro dataset possiede 506 osservazioni mi aspetto che:

- Il **training set** avrà 303 osservazioni.
- Il **validation set** avrà 101 osservazioni.
- Il **test set** avrà 101 osservazioni.

In reatà il test set avrà 102 osservazioni per via delle approssimazioni.



In [24]:
# Divisione del dataset

train_porzione = 0.6  
val_porzione = 0.2  
test_porzione = 0.2

train_dim= int(X.shape[0]*train_porzione)
val_dim= int(X.shape[0]*val_porzione)
X_train=X[:train_dim]
Y_train=y[:train_dim]
X_valadation=X[train_dim:train_dim+val_dim]
Y_valadation=y[train_dim:train_dim+val_dim]

X_test=X[train_dim+val_dim:]
Y_test=y[train_dim+val_dim:]

print("train shape: ", X_train.shape)
print("test shape: ", X_valadation.shape)
print("valadation shape: ", X_test.shape)

train shape:  (303, 13)
test shape:  (101, 13)
valadation shape:  (102, 13)


### **Esercizio 1: Costruisci una Pipeline di Regressione Lineare Standardizzata**

**Step 1:** Standardizza i dataset di addestramento, validazione e test. Usa `StandardScaler` di scikit-learn.  

**Step 2:** Aggiungi una feature costante (bias) ai dati concatenando una colonna di uno ad ogni dataset.  

**Step 3:** Implementa la soluzione in forma chiusa per l'addestramento di un modello di regressione lineare. 
 
**Step 4:** Valuta il modello calcolando il Mean Absolute Error (MAE) sui dataset di addestramento, validazione e test.


### **Guida**

1. **StandardScaler**:
   - Utilizza `StandardScaler` da `sklearn.preprocessing` per standardizzare i dati.
   - Il metodo `fit_transform` calcola la media e la varianza dei dati di addestramento e li scala di conseguenza.
   - Utilizza `transform` per standardizzare i dati di validazione e test utilizzando gli stessi parametri. Utilizziamo il metodo `transform` perchè non calcola i parametri di scaling (media e std). In questo modo ci assicuriamo che i dati di training e quelli di validation e test vengano scalati in modo uguale. Se usassimo `fit_transform` avremmo degli scaling diversi.

2. **Aggiunta di una Caratteristica Costante**:
   - Utilizza `np.c_` per concatenare una colonna di uno alle matrici delle caratteristiche. Questo è importante per includere il termine di intercetta nella regressione lineare.

3. **Soluzione in Forma Chiusa per la Regressione Lineare**:
   - La soluzione in forma chiusa è:

     $$\theta = (X^T X)^{-1} X^T y$$

   - Per calcolare la trasposta di una matrice possiamo utilizzare l' attributo `.T` di cui ogni array è dotato.

   - Utilizza `np.linalg.inv` di NumPy per l'inversione della matrice e l'operatore `@` per la moltiplicazione matriciale.
  
   - Puoi utilizzare l'operatore @ per eseguire l'operazione np.dot (`A @ B` è equivalente a `np.dot(A, B)`).

4. **Mean Absolute Error (MAE)**:
   - L'MAE si calcola come:

     $$\text{MAE} = \frac{1}{n} \sum_{i=1}^n |y_i - \hat{y}_i|$$

   - Utilizza `np.mean` e `np.abs` per calcolarlo.


In [25]:
# Step 1 - Normalizzazione dei dati. Dobbiamo normalizzare le features 
# sia del training set, validation set e test set.
from sklearn.preprocessing import StandardScaler


# Utilizziamo il metodo .fit_transform() dello scaler per normalizzare le feature di training.

# Per normalizzare le feature di validation e test utilizziamo il metodo .transform()
scaler = StandardScaler()

x_train_norm=scaler.fit_transform(X_train)
x_val_norm=scaler.transform(X_valadation)
x_test_norm=scaler.transform(X_test)

print(x_train_norm)
print(x_val_norm)
print(x_test_norm)
# svolgimento...

[[-0.44804325 -0.47722561 -1.20888589 ... -0.72714928  0.26235409
  -0.7767965 ]
 [ 0.77103386 -0.47722561  1.02487948 ...  0.80741501  0.17182767
  -0.46725622]
 [-0.44179028 -0.47722561  0.3920294  ... -0.95965902  0.4475036
  -0.33015391]
 ...
 [-0.44947979  1.23173418 -0.73236932 ... -0.40163564  0.47907794
  -1.31531178]
 [ 0.05336461 -0.47722561  1.02487948 ...  0.80741501  0.01695355
   0.19140026]
 [-0.44583123 -0.47722561  0.23268947 ...  0.10988579  0.47068476
  -0.52803354]]
[[-0.41465286 -0.47722561 -0.20775013 ... -0.02962005  0.41712829
   0.7765586 ]
 [-0.40351297 -0.47722561 -0.20775013 ... -0.02962005  0.46219166
  -0.34994187]
 [-0.37322047 -0.47722561 -0.47231454 ...  1.1794306   0.47907794
  -0.64676132]
 ...
 [-0.40213495 -0.47722561 -0.76393666 ... -0.49463953  0.23107951
  -0.91531224]
 [-0.14693384 -0.47722561  1.24735409 ... -1.75019213  0.08030206
   2.1885711 ]
 [ 0.08802262 -0.47722561  1.24735409 ... -1.75019213  0.47907794
   1.92002018]]
[[-0.45201503 -0.

In [26]:
# Step 2 - Aggiunta di una feature costante

# creiamo un vettore di 1 da aggiungere come feature costante. 
# ATTENZIONE: questo vettore deve avere le stesse righe del set a cui viene aggiunto. 
# Uno uguale per tutti non va bene

colonne_train=np.ones(x_train_norm.shape[0])
colonne_val=np.ones(x_val_norm.shape[0])
colonne_test=np.ones(x_test_norm.shape[0])

train_x=np.c_[colonne_train,x_train_norm]
val_x=np.c_[colonne_val,x_val_norm]
test_x=np.c_[colonne_test,x_test_norm]

print(train_x.shape)
print(val_x.shape)
print(test_x.shape)




(303, 14)
(101, 14)
(102, 14)


In [27]:
# Step 3 - Applichiamo la formula matematica della regressione lineare

# ATTENZIONE: stiamo per effettuare operazioni tra matrici e vettori, 
# non si tratta di una semplice formula matematica, stiamo attenti a quali operatori utilizzare e quanto
#Per calcolare la trasposta di una matrice possiamo utilizzare l' attributo .T di cui ogni array è dotato.

#Utilizza np.linalg.inv di NumPy per l'inversione della matrice e l'operatore @ per la moltiplicazione matriciale.

#Puoi utilizzare l'operatore @ per eseguire l'operazione np.dot (A @ B è equivalente a np.dot(A, B)).

x_train_trasposta=train_x.T
k=np.linalg.inv(x_train_trasposta@train_x)@x_train_trasposta@Y_train

print(k)
y_train_pred=train_x@k
y_test_pred=test_x@k
y_val_pred=val_x@k
print(y_train_pred.shape)
print(y_test_pred.shape)
print(y_val_pred.shape)

[22.16138614 -0.50926449  1.04570167  0.08640982  0.43014273 -1.8975044
  2.62964478 -0.02436637 -2.95441032  2.92892785 -2.41571656 -1.74144698
  1.12132994 -3.56543566]
(303,)
(102,)
(101,)


In [28]:
# Step 4 - Calcolo MAE

# Calcoliamo l'errore medio assoluto (MAE) per il training set, validation set e test set.
# Utlizziamo la formula specificata nella guida.
#Utilizza np.mean e np.abs per calcolarlo.


m_train=np.mean(np.abs(y_train_pred-Y_train))
m_val=np.mean(np.abs(y_val_pred-Y_valadation))
m_test=np.mean(np.abs(y_test_pred-Y_test))

print(m_train)
print(m_val)
print(m_test)

3.2496837441036983
3.5977784744283965
3.0197366706166866


### **Esercizio: Costruisci una pipeline di Regressione Lineare Standardizzata utilizzando `scikit-learn`** 

**Step 1 & 2:** Step 1 e 2 sono uguali a quanto fatto prima.

**Step 3:** Utilizza `LinearRegression()` di scikit-learn per addestrare un modello di regressione lineare.  

**Step 4:** Valuta il modello calcolando il Mean Absolute Error (MAE) sui dataset di addestramento, validazione e test, utilizzando `mean_absolute_error()` da `sklearn.metrics`.


## `LinearRegression` da Scikit-Learn

La classe `LinearRegression` in Scikit-Learn viene utilizzata per eseguire la **regressione lineare**, adattando un modello lineare al dataset.

## **Sintassi**
```python
from sklearn.linear_model import LinearRegression

model = LinearRegression()
# Dati di esempio
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.array([10, 15, 20, 25])

# Adatta il modello ai dati
model.fit(X, y)

# Predici nuovi valori
X_new = np.array([[3, 5], [5, 9]])
predictions = model.predict(X_new)


## `mean_absolute_error` da Scikit-Learn

La funzione `mean_absolute_error` calcola l'**errore assoluto medio** (MAE) tra i valori target reali e quelli predetti.

## **Sintassi**
```python
sklearn.metrics.mean_absolute_error(y_true, y_pred)


### **Guida**

1. **Istanziare e allenare un modello di regressione lineare**:
    
    - Istanziamo una classe `LinearRegression` per creare il modello.
    - Utilizziamo il metodo `.fit()` per allenare il modello con i dati di training.

2. **Effettuare predizioni con il modello**:

    - Utiliziamo il metodo `.predict()` del modello per effettuare le predizioni. Effettuiamo le predizioni per tutti i set che abbiamo (train, validation e test).

3. **Calcolo della MAE**: 

    - Calcolare MAE su tutti i set utilizzando la funzione `mean_abslute_error`


In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [30]:
# Step 1 - Istanziare e allenare il modello di regressione lineare.
regressione_lineare=LinearRegression()

regressione_lineare.fit(train_x,Y_train)


LinearRegression()

In [31]:
# Step 2 - Effettuare predizioni
y_pred_train=regressione_lineare.predict(train_x)
y_pred_val=regressione_lineare.predict(val_x)
y_pred_test=regressione_lineare.predict(test_x)

print(y_pred_train.shape)
print(y_pred_val.shape)
print(y_pred_test.shape)

(303,)
(101,)
(102,)


In [32]:
# Step 3 - Calcolo MAE

mae_train=mean_abslute_error (y_train,y_pred_train)
mae_val=mean_abslute_error (y_valadation,y_pred_val)
mae_test=mean_abslute_error(y_test,y_pred_test)

NameError: name 'mean_abslute_error' is not defined

### **Esercizio: Crea una funzione che esegua una pipeline di Regressione Lineare**

La funzione deve richiedere un parametro `hyperparams` per gestire i diversi casi. 

`hyperparams` deve essere un dizionario contenente diverse chiavi, in base al valore di queste chiavi devono essere eseguiti (oppure no) diversi pezzi di codice. 

In questo esercizio la chiave da utilizzare sarà `hyperparams['data_standardize']`. Se il valore di questa chiave sarà **True** allora eseguire la standardizzazione con `scikit-learn`, se invece è **False** non verrà eseguita alcuna standardizzazione.

**Step 1:** Controllare se eseguire o no la standardizzazione.

* **Step 1.1:** Scrivere il codice per eseguire la standardizzazione.

**Step 2:** Utilizza `np.c_` per concatenare una colonna di uno alle matrici delle caratteristiche.

**Step 3:** Applichiamo la formula matematica della regressione lineare.

**Step 4:** Calcolo MAE utilizzando la formula (NON con `scikit-learn`).

La funzione deve ritornare i valori della MAE.

Dopo aver testato i risultati con `hyperparams['data_standardize']` = **True**, provare anche i risultati ottenuti se `hyperparams['data_standardize']` = **False**.

In [ ]:
# svolgimento...

def pipeline(X_train, y_train, X_val, y_val, hyperparams):

    X_train = np.array(X_train, dtype=float)
    y_train = np.array(y_train, dtype=float)
    X_val = np.array(X_val, dtype=float)
    y_val = np.array(y_val, dtype=float)
    
    # Step 1 - Controllo se è richiesta la standardizzazione dei dati
      if hyperparams['data_standardize']:  
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Step 2 - Concatenare una colonna di uno alla matrice delle features (per il termine di bias)
    X_train = np.c_[np.ones(X_train.shape[0]), X_train]
    X_val = np.c_[np.ones(X_val.shape[0]), X_val]

    # Step 3 - Applicare la formula della regressione lineare e calcolare predizioni
    theta = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

    # Predizioni sui dati di validazione
    y_pred = X_val @ theta

    # Step 4 - Calcolo MAE manualmente
    mae = np.mean(np.abs(y_pred - y_val))

    return mae


In [ ]:
hyperparams = {'data_standardize': True}

train_fraction = 0.8
validation_fraction = 0.2

num_train = int(train_fraction * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_validation = X[num_train:]
y_validation = y[num_train:]

# Chiamare la funzione pipeline e stampare i risultati della MAE
mae_standardized = pipeline(X_train, y_train, X_validation, y_validation, hyperparams)
print("MAE con standardizzazione:", mae_standardized)
mae_standardized = pipeline(X_train, y_train, X_validation, y_validation, hyperparams)
print("MAE con standardizzazione:", mae_standardized)


### **Esercizio: Implementare alla funzione `pipeline` la possibilità di usare PCA**

Modifichiamo la funzione `pipeline` in modo da gestire anche la possibilità di effettuare la PCA. Dunque aggiungiamo al dizionario `hyperparams` la chiave `use_pca`. 

Se `hyperparams['use_pca']` = **True** verrà eseguita la PCA. 

Se `hyperparams['use_pca']` = **False** non verrà eseguita la PCA.

La gestione della standardizzazione deve essere mantenuta come prima.

In [ ]:
# svolgimento...
from sklearn.decomposition import PCA

def pipeline(X_train, y_train, X_val, y_val, hyperparams):

    X_train = np.array(X_train, dtype=float)
    y_train = np.array(y_train, dtype=float)
    X_val = np.array(X_val, dtype=float)
    y_val = np.array(y_val, dtype=float)

    # Step 1 - Controllo se è richista la PCA
    if hyperparams['use_pca']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)  
        # Step 1.1 - Scrivere il codice per applicare PCA
    
    # Step 2 - Controllo se è richiesta la standardizzazione dei dati
    if hyperparams['data_standardize']:
        n_components = hyperparams.get('pca_components', min(X_train.shape[1], X_train.shape[0]))  # Numero di componenti
        pca = PCA(n_components=n_components)
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)
        # Step 2.1 - Scrivere il codice per standardizzare i dati 


    # Step 3 - Concatenare una colonna di uno alla matrice delle features
    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]  
    X_val = np.c_[np.ones((X_val.shape[0], 1)), X_val] 
    # Step 4 - Applicare formula della regressione lineare e calcolare predizioni
    theta = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train 
    # Step 5 - Calcolare MAE 
    mae = np.mean(np.abs(y_pred - y_val))  

    return mae

In [ ]:
hyperparams = {'data_standardize': True, 'use_pca': True}
train_fraction = 0.8
validation_fraction = 0.2

num_train = int(train_fraction * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_validation = X[num_train:]
y_validation = y[num_train:]

# Chiamare la funzione pipeline e stampare i risultati della MAE al variare dell' utilizzo della PCA.
hyperparams_pca = {'data_standardize': True, 'use_pca': True, 'pca_components': 2}
mae_pca = pipeline(X_train, y_train, X_validation, y_validation, hyperparams_pca)
print("MAE con PCA:", mae_pca)

hyperparams_no_pca = {'data_standardize': True, 'use_pca': False}
mae_no_pca = pipeline(X_train, y_train, X_validation, y_validation, hyperparams_no_pca)
print("MAE senza PCA:", mae_no_pca)